In [ ]:
import os
import json
import pandas as pd
import numpy as np
import datetime

In [ ]:
sample_open_data_ath = "E:\gc_opendata\9787e81f-19f6-4cc1-891d-63e68b4d2f9f"

In [ ]:
class athlete(object):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        summary_filename = '{%s}.json' % (root_dir.split('/')[-1].split('\\')[-1])
        self.athlete_summary_file = os.path.join(root_dir,summary_filename)

        self.athlete_files = next(os.walk(root_dir), (None, None, []))[2]
        [self.athlete_files.remove(file) if '.csv' not in file else None for file in self.athlete_files]
        self.athlete_activity_filepaths = [os.path.join(root_dir,filepath) for filepath in self.athlete_files]
    
    @staticmethod
    def calc_xspeed(x, secs_id, dist_id, elev_id):
        xspeed = (
            1+9*(                                  # slope adj
                (x[elev_id,1:] - x[elev_id,:-1]) / # delta alt
                (x[dist_id,1:] - x[dist_id,:-1])   # delta dist
            )
        ) * (
                (x[dist_id,1:] - x[dist_id,:-1]) / # delta dist
                (x[secs_id,1:] - x[secs_id,:-1])   # delta secs
                * (60*60)                          # convert to kph
            )                                      # slope running speed
        return xspeed
    
    def get_activity_metadata(self, return_metadata=False):
        self.activity_metadata = []
        with open(os.path.join(sample_open_data_ath,'{9787e81f-19f6-4cc1-891d-63e68b4d2f9f}.json'),'r') as f:
            ath_over = f.read()
        f.close()
        ath_over = json.loads(ath_over)
        for activity in ath_over['RIDES']:
            date_obj = datetime.datetime.strptime(activity['date'], '%Y/%m/%d %H:%M:%S %Z')
            act_metadata = {'sport':activity['sport']
                            ,'datetime':date_obj}
            try:
                act_metadata.update({'cp_setting':float(activity['METRICS']['cp_setting'])})
            except:
                act_metadata.update({'cp_setting':0})
            self.activity_metadata.append(act_metadata)
        if return_metadata:
            return self.activity_metadata
        else:
            return "Metadata processed"

    def extract_activity(self, activity_filepath, metrics=['secs','km','hr','power','alt'], make_xspeed=False):
        if make_xspeed and 'alt' not in metrics:
            metrics.append('alt')
        
        assert activity_filepath in self.athlete_activity_filepaths, "activity unavalable in dir"
        df = pd.read_csv(activity_filepath, engine='pyarrow')
        activity_array = df[metrics].to_numpy().T
        if make_xspeed:
            try:
                xspeed = self.calc_xspeed(activity_array, metrics.index('secs'), metrics.index('km'), metrics.index('alt'))
                xspeed = np.concatenate((np.array([0]),xspeed))
            except:
                xspeed = np.zeros(shape=activity_array.shape[1])
            activity_array = np.concatenate((activity_array,[xspeed]), axis=0)
        return activity_array

In [ ]:
ath0 = athlete(root_dir=sample_open_data_ath)
ath0.get_activity_metadata()


In [ ]:
max_array = 0
dim_array = 0
all_activity_arrays = []
for activity_filepath in ath0.athlete_activity_filepaths:
    activity_array = ath0.extract_activity(activity_filepath=activity_filepath, make_xspeed=True)
    if activity_array.shape[1] > max_array:
        max_array = activity_array.shape[1]
    if activity_array.shape[0] > dim_array:
        dim_array = activity_array.shape[0]
    activity_datetime_str = activity_filepath.split('\\')[-1]
    activity_datetime = datetime.datetime.strptime(activity_datetime_str, '%Y_%m_%d_%H_%M_%S.csv')
    all_activity_arrays.append({'activity_dt':activity_datetime,'data_array':activity_array})

In [ ]:
for activ_array in all_activity_arrays:
    activ_array

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.input = nn.Linear(in_features=input_dim, out_features=hidden_dim)
        self.hidden_1 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.output = nn.Linear(in_features=16, out_features=output_dim)
        
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden_1(x))
        return self.output(x)
    
    
model = Net()
print(model)

In [ ]:
num_epochs = 200
train_accuracies, test_accuracies = [], []

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    # Train set
    for X, y in train_loader:
        preds = model(X)
        pred_labels = torch.argmax(preds, axis=1)
        loss = loss_function(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_accuracies.append(
        100 * torch.mean((pred_labels == y).float()).item()
    )
    
    # Test set
    X, y = next(iter(test_loader))
    pred_labels = torch.argmax(model(X), axis=1)
    test_accuracies.append(
        100 * torch.mean((pred_labels == y).float()).item()
    )